In [1]:
import os
import sys
HOME_DIR = 'E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering'
os.chdir(HOME_DIR)

DATASETS_DIR = f'{HOME_DIR}/datasets'
DATA_DIR = f'{DATASETS_DIR}/08'
W1_FILE = f'{DATA_DIR}/ampc/w1.csv'
W2_FILE = f'{DATA_DIR}/ampc/w2.csv'
W3_FILE = f'{DATA_DIR}/ampc/w3.csv'
W4_FILE = f'{DATA_DIR}/ampc/w4.csv'

print(f'HOME_DIR: {HOME_DIR}')
print(f'DATASETS_DIR: {DATASETS_DIR}')
print(f'DATA_DIR: {DATA_DIR}')
print(f'W1_FILE: {W1_FILE}')
print(f'W2_FILE: {W2_FILE}')
print(f'W3_FILE: {W3_FILE}')
print(f'W4_FILE: {W4_FILE}')

HOME_DIR: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering
DATASETS_DIR: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering/datasets
DATA_DIR: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering/datasets/08
W1_FILE: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering/datasets/08/ampc/w1.csv
W2_FILE: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering/datasets/08/ampc/w2.csv
W3_FILE: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering/datasets/08/ampc/w3.csv
W4_FILE: E:\#Projects\COS40007-Artificial-Intelligence-for-Engineering/datasets/08/ampc/w4.csv


In [2]:
!pip install --upgrade pip --quiet
!pip install -r requirements.txt --quiet

Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "e:\#Projects\COS40007-Artificial-Intelligence-for-Engineering\venv\Scripts\pip.exe\__main__.py", line 4, in <module>
ModuleNotFoundError: No module named 'pip'
Traceback (most recent call last):
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Program Files\WindowsApps\PythonSoftwareFoundation.Python.3.9_3.9.3568.0_x64__qbz5n2kfra8p0\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "e:\#Projects\COS40007-

In [3]:
import math
import json
import pandas as pd
import numpy as np

from scipy.integrate import simpson as simps
from scipy.signal import find_peaks

from sklearn import svm
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.decomposition import PCA

import matplotlib.pyplot as plt
import seaborn as sns

# Combine CSV

In [4]:
# Load the CSV files
w1 = pd.read_csv(W1_FILE)
w2 = pd.read_csv(W2_FILE)
w3 = pd.read_csv(W3_FILE)
w4 = pd.read_csv(W4_FILE)

# Combine the datasets
combined_data = pd.concat([w1, w2, w3, w4])

# Save the combined data to a new CSV file
combined_data_file = f'{DATA_DIR}/combined_data.csv'
combined_data.to_csv(combined_data_file, index=False)

print(f'Combined data saved to: {combined_data_file}')
print(combined_data.head())


In [ ]:
# Display basic information about the dataset
print("Dataset Information:")
print(combined_data.info())

# Check the shape of the dataset
print(f"Dataset Shape: {combined_data.shape}")

# Summary statistics
print("Summary Statistics:")
print(combined_data.describe())

In [ ]:
# Check for missing values
missing_values = combined_data.isnull().sum()
print("Missing Values:")
print(missing_values[missing_values > 0])

In [ ]:
# Distribution of the target variable (last column)
class_distribution = combined_data.iloc[:, -1].value_counts()
print("Class Distribution:")
print(class_distribution)

# Plot the class distribution
sns.countplot(x=combined_data.columns[-1], data=combined_data)
plt.title('Class Distribution')
plt.show()

In [ ]:
# Compute the correlation matrix
correlation_matrix = combined_data.corr()

# Plot the heatmap
plt.figure(figsize=(12, 10))
sns.heatmap(correlation_matrix, cmap='coolwarm', annot=False, fmt=".2f")
plt.title('Correlation Matrix')
plt.show()

In [ ]:
# Plot the distribution of the first 5 features as an example
combined_data.iloc[:, :5].hist(bins=20, figsize=(15, 10))
plt.suptitle("Distribution of First 5 Features")
plt.show()

In [ ]:
# Plot boxplots to check for outliers in the first 5 features
plt.figure(figsize=(15, 10))
for i in range(5):
    plt.subplot(2, 3, i+1)
    sns.boxplot(data=combined_data.iloc[:, i])
    plt.title(f'Feature {i+1} Boxplot')
plt.tight_layout()
plt.show()

In [ ]:
# Check variance of all features
feature_variances = combined_data.var()
print("Variance of Features:")
print(feature_variances.sort_values(ascending=False).head(10))  # Display top 10 features with highest variance

In [ ]:
# Correlation of each feature with the target variable
target_corr = correlation_matrix.iloc[:, -1].sort_values(ascending=False)
print("Correlation with Target Variable:")
print(target_corr)

# Shuffle the Data and Save to all_data.csv

In [ ]:
# Shuffle the combined dataset
shuffled_data = combined_data.sample(frac=1, random_state=1).reset_index(drop=True)

# Save the shuffled data to a new CSV file
all_data_file = f'{DATA_DIR}/all_data.csv'
shuffled_data.to_csv(all_data_file, index=False)

print(f'Shuffled data saved to: {all_data_file}')

In [ ]:
# NOTE: DATASET TOO LARGE AND TAKES FOREVER TO RUN ON THE MACHINE SO INSTEAD OF TAKING ALL DATA, WE WILL TAKE A SAMPLE OF THE DATA FOR TRAINING AND TESTING

# Shuffle the combined dataset
shuffled_data = combined_data.sample(frac=1, random_state=1).reset_index(drop=True)

# Sample 1% of the shuffled data
sampled_data = shuffled_data.sample(frac=0.01, random_state=1)

# Save the sampled and shuffled data to a new CSV file
sampled_data_file = f'{DATA_DIR}/sampled_all_data.csv'
sampled_data.to_csv(sampled_data_file, index=False)

print(f'Sampled and shuffled data saved to: {sampled_data_file}')


# Train the SVM Model with Train-Test Split

In [ ]:
# Load the data
all_data = pd.read_csv(sampled_data_file)

# Separate features and labels
X = all_data.iloc[:, :-1]  # All columns except the last one
y = all_data.iloc[:, -1]  # The last column

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Train the SVM model
clf = svm.SVC(kernel='linear')
clf.fit(X_train, y_train)

# Predict on the test set
y_pred = clf.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy on test set: {accuracy:.2f}')


# Perform 10-Fold Cross-Validation

In [ ]:
# Perform cross-validation
scores = cross_val_score(clf, X, y, cv=10)

print(f'Cross-validation accuracy: {scores.mean():.2f}')


# Hyperparameter Tuning using GridSearchCV

In [ ]:
# Define the parameter grid
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf', 'poly'], 'gamma': [0.1, 1, 10]}

# Initialize GridSearchCV
grid_search = GridSearchCV(svm.SVC(), param_grid, cv=10)
grid_search.fit(X_train, y_train)

# Print best parameters and accuracy
print(f'Best parameters: {grid_search.best_params_}')
print(f'Best cross-validation accuracy: {grid_search.best_score_:.2f}')

# Retrain the SVM Model with Optimal Hyperparameters

In [ ]:
# Retrain the model using the best parameters
best_clf = grid_search.best_estimator_

# Train and test split accuracy
best_clf.fit(X_train, y_train)
y_pred_best = best_clf.predict(X_test)
accuracy_best = accuracy_score(y_test, y_pred_best)
print(f'Accuracy with tuned parameters on test set: {accuracy_best:.2f}')

# Cross-validation accuracy with tuned parameters
scores_best = cross_val_score(best_clf, X, y, cv=10)
print(f'Cross-validation accuracy with tuned parameters: {scores_best.mean():.2f}')

# Feature Selection using K-Best

In [ ]:
# Select the top 100 features
k_best = SelectKBest(f_classif, k=100)
X_k_best = k_best.fit_transform(X, y)

# Train and evaluate with the selected features
X_train_k, X_test_k, y_train_k, y_test_k = train_test_split(X_k_best, y, test_size=0.3, random_state=1)
clf_k_best = svm.SVC(kernel='linear')
clf_k_best.fit(X_train_k, y_train_k)
y_pred_k_best = clf_k_best.predict(X_test_k)
accuracy_k_best = accuracy_score(y_test_k, y_pred_k_best)
print(f'Accuracy with K-Best features on test set: {accuracy_k_best:.2f}')

# Cross-validation accuracy with K-Best features
scores_k_best = cross_val_score(clf_k_best, X_k_best, y, cv=10)
print(f'Cross-validation accuracy with K-Best features: {scores_k_best.mean():.2f}')

# Dimensionality Reduction using PCA

In [ ]:
# Apply PCA and select top 10 components
pca = PCA(n_components=10)
X_pca = pca.fit_transform(X)

# Train and evaluate with the PCA features
X_train_pca, X_test_pca, y_train_pca, y_test_pca = train_test_split(X_pca, y, test_size=0.3, random_state=1)
clf_pca = svm.SVC(kernel='linear')
clf_pca.fit(X_train_pca, y_train_pca)
y_pred_pca = clf_pca.predict(X_test_pca)
accuracy_pca = accuracy_score(y_test_pca, y_pred_pca)
print(f'Accuracy with PCA features on test set: {accuracy_pca:.2f}')

# Cross-validation accuracy with PCA features
scores_pca = cross_val_score(clf_pca, X_pca, y, cv=10)
print(f'Cross-validation accuracy with PCA features: {scores_pca.mean():.2f}')

# Summary

In [ ]:
summary_data = {
    'Model': ['Original features', 'With hyperparameter tuning', 'With K-Best features', 'With PCA features'],
    'Train-test split accuracy': [accuracy, accuracy_best, accuracy_k_best, accuracy_pca],
    'Cross-validation accuracy': [scores.mean(), scores_best.mean(), scores_k_best.mean(), scores_pca.mean()]
}

summary_table = pd.DataFrame(summary_data)
print(summary_table)